In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

import pickle
import sys
import matplotlib.pyplot as plt
# just set the seed for the random number generator
np.random.seed(107)

import numpy as np
np.random.seed(1) # NumPy
import random
random.seed(3) # Python
import tensorflow as tf

from keras import backend as K

import matplotlib.cm as cm

from sklearn.manifold import TSNE

from keras.layers import Dense, Flatten, LSTM, Dropout

In [2]:
# Usando Google Colab:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Python/MEU_TCC/Classes')
pasta = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
print(pasta)

preços = pd.read_csv(pasta + "Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)


Mounted at /content/drive
/content/drive/MyDrive/Python/MEU_TCC/Data/


In [3]:
%load_ext autoreload
%aimport class_SeriesAnalyser, class_Trader, class_DataProcessor
%autoreload 1

series_analyser = class_SeriesAnalyser.SeriesAnalyser()
trader = class_Trader.Trader()
data_processor = class_DataProcessor.DataProcessor()

In [4]:
top_n = 10
subsample = 2500
min_half_life = 0 # number of points in a day
max_half_life = 20000 #~number of points in a year: 78*252
file_extension = 'intraday'

# Calculando Renda fixa anualizada:

In [5]:
pastaC = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
CDI_ret = pd.read_csv(pastaC + "CDI_ret.csv", )
# Convert the 'data' column to datetime format
CDI_ret.index = pd.to_datetime(CDI_ret['Data'])
CDI_ret.drop(['Data'],inplace=True,axis=1)

CDI_ret = CDI_ret[1:]

In [6]:
anoi = 1996
rfannualized = pd.DataFrame(columns=['retorno_anualizado'])
inicio = 0

for nsem in range(0, len(semestres) - 2,2):
    print(f"Starting period {nsem} | Past days: {inicio}")
    ano = math.floor(anoi + nsem/2)
    print(ano)

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][nsem]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]

    twelve_months = (Datas == semestres['Data'][nsem + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_months][0]
    print('inicio: ',inicio,' twelve_months: ',twelve_months)


    annualized_ret = (np.cumprod(1 + CDI_ret['CDI Acumulado'][inicio:twelve_months]) - 1)[-1]
    rfannualized.loc[ano] = annualized_ret

rf_mean_ano = rfannualized.mean()[0]

Starting period 0 | Past days: 0
1996
inicio:  1996-01-02 00:00:00  twelve_months:  1996-12-31 00:00:00
Starting period 2 | Past days: 1996-01-02 00:00:00
1997
inicio:  1996-12-31 00:00:00  twelve_months:  1997-12-31 00:00:00
Starting period 4 | Past days: 1996-12-31 00:00:00
1998
inicio:  1997-12-31 00:00:00  twelve_months:  1998-12-31 00:00:00
Starting period 6 | Past days: 1997-12-31 00:00:00
1999
inicio:  1998-12-31 00:00:00  twelve_months:  1999-12-31 00:00:00
Starting period 8 | Past days: 1998-12-31 00:00:00
2000
inicio:  1999-12-31 00:00:00  twelve_months:  2000-12-29 00:00:00
Starting period 10 | Past days: 1999-12-31 00:00:00
2001
inicio:  2000-12-29 00:00:00  twelve_months:  2001-12-31 00:00:00
Starting period 12 | Past days: 2000-12-29 00:00:00
2002
inicio:  2001-12-31 00:00:00  twelve_months:  2002-12-31 00:00:00
Starting period 14 | Past days: 2001-12-31 00:00:00
2003
inicio:  2002-12-31 00:00:00  twelve_months:  2003-12-31 00:00:00
Starting period 16 | Past days: 2002-12

#No Cluster

In [ ]:
inicio = 0
pairs_t = []
performance_threshold_test_t = []
sharpe_results_threshold_test_nocosts_t = []
cum_returns_threshold_test_nocosts_t = []
results_t = pd.DataFrame(columns=['n_pairs','Daily_sharpe_ratio','avg_annual_roi',
                                  'Total Number of Trades','Positive Trades','Negative Trades','pct_positive_trades_per_pair',
                                  'pct_pairs_with_positive_returns','avg_half_life','avg_hurst_exponent','Maximum Drawdown'])

Pares_Total = pd.DataFrame(columns=['Ticker 1','Ticker 2','p valor'])

for big_loop in range(0, len(semestres) - 3):
    print(f"\nStarting period {big_loop} | Past days: {inicio}")

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]
    twelve_month = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_month][0]
    inicio_test = Datas.iloc[twelve_month+1][0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]
    six_months = Datas.iloc[six_months][0]

    # split data in training and test
    df_prices_train, df_prices_test = data_processor.split_data(preços,(inicio, twelve_months),(inicio_test,six_months),remove_nan=True)

    # interday
    n_years_test = round(len(df_prices_train)/(250))

    no_cluster_series = pd.Series(0,index = df_prices_train.columns)
    pairs_all_a_all, unique_tickers = series_analyser.get_candidate_pairs(clustered_series=no_cluster_series,
                                                                          pricing_df_train=df_prices_train,
                                                                          pricing_df_test=df_prices_test,
                                                                          min_half_life=min_half_life,
                                                                          max_half_life=max_half_life,
                                                                          min_zero_crosings=12,
                                                                          p_value_threshold=0.01,
                                                                          hurst_threshold=0.5,
                                                                          subsample=0,
                                                                          top_n=top_n
                                                                          )
    print('Out of {} possible combinations'.format(len(df_prices_train.columns)*(len(df_prices_train.columns)-1)/2))

    results_without_costs, results_with_costs, performance_threshold_test = \
            trader.apply_trading_strategy(pairs_all_a_all,
                                          strategy='fixed_beta',
                                          entry_multiplier=2,
                                          exit_multiplier=0,
                                          test_mode=True,
                                          train_val_split=0,
                                          rf = rfannualized
                                          )

    sharpe_results_threshold_test_nocosts, cum_returns_threshold_test_nocosts = results_without_costs
    #sharpe_results_threshold_test_w_costs, cum_returns_threshold_test_w_costs = results_with_costs

    '''
    pairs_t.extend(pairs_all_a_all)
    sharpe_results_threshold_test_nocosts_t.extend(sharpe_results_threshold_test_nocosts)
    cum_returns_threshold_test_nocosts_t.extend(cum_returns_threshold_test_nocosts)
    performance_threshold_test_t.extend(performance_threshold_test)
    '''

    # interday
    n_years_test = round(len(df_prices_test)/(240))

    n_years_test

    results, pairs_summary = trader.summarize_results(sharpe_results_threshold_test_nocosts,
                                                      cum_returns_threshold_test_nocosts,
                                                      performance_threshold_test,
                                                      pairs_all_a_all,
                                                      n_years_test,
                                                      rf = rfannualized)

    for i in range(0, len(pairs_all_a_all)):
      Pares_Total = Pares_Total.append({'Ticker 1': pairs_all_a_all[i][0], 'Ticker 2': pairs_all_a_all[i][1], 'p valor': pairs_all_a_all[i][2]['p_value']},ignore_index=True)

    results_t = results_t.append(results,ignore_index=True)

results_t['Annualized SR'] = results_t['Daily_sharpe_ratio']*(252**0.5)

Pares_Total['Ticker Combinado'] = Pares_Total[['Ticker 1', 'Ticker 2']].apply(lambda x: '-'.join(sorted(x)), axis=1)
Pares_Total = Pares_Total.drop_duplicates(subset=['Ticker Combinado'], keep='first')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
pd.set_option('display.float_format', '{:.10e}'.format)


In [ ]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Cointegrados_Fixed_Beta_10.pickle', 'wb') as f:
    pickle.dump(results_t, f)

with open(pastaP + 'Pares_Cointegrados_Lista_10.pickle', 'wb') as f:
    pickle.dump(Pares_Total, f)

In [7]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Cointegrados_Fixed_Beta_10.pickle', 'rb') as handle:
    results_t = pickle.load(handle)

with open(pastaP + 'Pares_Cointegrados_Lista_10.pickle', 'rb') as handle:
    Pares_Total = pickle.load(handle)

In [8]:
#Sharpe Ratio Anualizado:
SRA = (results_t['avg_annual_roi'].mean())/np.std(results_t['avg_annual_roi'])

In [9]:
# Resultados:
print('Top N Pares:',results_t['n_pairs'].min())
#print('Sharpe Ratio Anualizado: {:.3f}'.format(results_t['Annualized SR'].mean()))
print('Sharpe Ratio Anualizado: {:.3f}'.format(SRA))
print('Excesso de Retornos Anualizado: {:.3f} %'.format(results_t['avg_annual_roi'].mean()))
print('MDD: {:.3f} %'.format(results_t['Maximum Drawdown'].min()))
print('Total de Pares:', Pares_Total['Ticker Combinado'].count()) # Não está contando a repetição dos pares
print('Pares rentáveis: {:.3f} %'.format(results_t['pct_pairs_with_positive_returns'].mean()))
print('Total de Transações:', results_t['Total Number of Trades'].sum())
print('Operações rentáveis:', results_t['Positive Trades'].sum())
print('Operações deficitárias :', results_t['Negative Trades'].sum())

Top N Pares: 10
Sharpe Ratio Anualizado: 1.433
Excesso de Retornos Anualizado: 10.314 %
MDD: -38.632 %
Total de Pares: 500
Pares rentáveis: 61.667 %
Total de Transações: 616
Operações rentáveis: 465
Operações deficitárias : 151


# Tentando separar funcoes

In [ ]:
'''#def summarize_results(self, sharpe_results, cum_returns, performance, total_pairs, n_years,rf):
#(sharpe_results_threshold_test_nocosts_t, cum_returns_threshold_test_nocosts_t, performance_threshold_test_t, pairs_t, n_years, rf = rf_mean_ano)
sharpe_results = sharpe_results_threshold_test_nocosts_t
cum_returns = cum_returns_threshold_test_nocosts_t
performance = performance_threshold_test_t
total_pairs = pairs_t
rf = rf_mean_ano



avg_total_roi, avg_annual_roi, positive_pct = trader.calculate_metrics(cum_returns, n_years)

#portfolio_sharpe_ratio = trader.calculate_portfolio_sharpe_ratio(performance, total_pairs,rf)

sorted_indices = np.flip(np.argsort(sharpe_results), axis=0)
# print(sorted_indices)
# initialize list of lists
data = []
for index in sorted_indices:
    # get number of positive and negative positions
    position_returns = performance[index][1]['position_ret_with_costs']
    positive_positions = len(position_returns[position_returns > 0])
    negative_positions = len(position_returns[position_returns < 0])
    data.append([total_pairs[index][0],
                  total_pairs[index][1],
                  total_pairs[index][2]['t_statistic'],
                  total_pairs[index][2]['p_value'],
                  total_pairs[index][2]['zero_cross'],
                  total_pairs[index][2]['half_life'],
                  total_pairs[index][2]['hurst_exponent'],
                  positive_positions,
                  negative_positions,
                  sharpe_results[index]
                  ])

# Create the pandas DataFrame
pairs_df = pd.DataFrame(data, columns=['Leg1', 'Leg2', 't_statistic', 'p_value',
                                        'zero_cross', 'half_life', 'hurst_exponent', 'positive_trades',
                                        'negative_trades', 'sharpe_result'])

pairs_df['positive_trades_per_pair_pct'] = (pairs_df['positive_trades']) / \
                                            (pairs_df['positive_trades'] + pairs_df['negative_trades']) * 100

print('Total number of trades: ', pairs_df.positive_trades.sum() + pairs_df.negative_trades.sum())
print('Positive trades: ', pairs_df.positive_trades.sum())
print('Negative trades: ', pairs_df.negative_trades.sum())

avg_positive_trades_per_pair_pct = pairs_df['positive_trades_per_pair_pct'].mean()

results = {'n_pairs': len(sharpe_results),
            #'portfolio_sharpe_ratio': portfolio_sharpe_ratio,
            'avg_total_roi': avg_total_roi,
            'avg_annual_roi': avg_annual_roi,
            'pct_positive_trades_per_pair': avg_positive_trades_per_pair_pct,
            'pct_pairs_with_positive_results': positive_pct,
            'avg_half_life': pairs_df['half_life'].mean(),
            'avg_hurst_exponent': pairs_df['hurst_exponent'].mean()}

# Drawdown info
total_account_balance = performance[0][1]['account_balance']
for index in range(1, len(total_pairs)):
    total_account_balance = total_account_balance + performance[index][1]['account_balance']
total_account_balance = total_account_balance.fillna(method='ffill')
max_dd, max_dd_duration, total_dd_duration = trader.calculate_maximum_drawdown(total_account_balance)
print('Maximum drawdown of portfolio: {:.2f}%'.format(max_dd))

#return results, pairs_df'''

"#def summarize_results(self, sharpe_results, cum_returns, performance, total_pairs, n_years,rf):\n#(sharpe_results_threshold_test_nocosts_t, cum_returns_threshold_test_nocosts_t, performance_threshold_test_t, pairs_t, n_years, rf = rf_mean_ano)\nsharpe_results = sharpe_results_threshold_test_nocosts_t\ncum_returns = cum_returns_threshold_test_nocosts_t\nperformance = performance_threshold_test_t\ntotal_pairs = pairs_t\nrf = rf_mean_ano\n\n\n\navg_total_roi, avg_annual_roi, positive_pct = trader.calculate_metrics(cum_returns, n_years)\n\n#portfolio_sharpe_ratio = trader.calculate_portfolio_sharpe_ratio(performance, total_pairs,rf)\n\nsorted_indices = np.flip(np.argsort(sharpe_results), axis=0)\n# print(sorted_indices)\n# initialize list of lists\ndata = []\nfor index in sorted_indices:\n    # get number of positive and negative positions\n    position_returns = performance[index][1]['position_ret_with_costs']\n    positive_positions = len(position_returns[position_returns > 0])\n    ne

In [ ]:
'''from datetime import timedelta

pairs = total_pairs


# calculate total daily account balance & df with returns
total_account_balance = performance[0][1]['account_balance'].resample('D').last().dropna()
portfolio_returns = total_account_balance.pct_change().fillna(0)
for index in range(1, len(pairs)):
    pair_balance = performance[index][1]['account_balance'].resample('D').last().dropna()
    total_account_balance = total_account_balance + pair_balance
    print(total_account_balance)  #ERRO
    portfolio_returns = pd.concat([portfolio_returns, pair_balance.pct_change().fillna(0)], axis=1)


# add first day with initial balance
total_account_balance = pd.Series(data=[len(pairs)],
                                  index=[total_account_balance.index[0] - timedelta(days=1)]).append(
                                  total_account_balance)

# calculate portfolio volatility
weights = np.array([1 / len(pairs)] * len(pairs))
vol = np.sqrt(np.dot(weights.T, np.dot(portfolio_returns.cov(), weights)))

annualized_ret = (total_account_balance[-1]-len(pairs))/len(pairs)
year = total_account_balance.index[-1].year

# assuming iid return's distributio, sr may be calculated as:
sharpe_ratio = (annualized_ret - rf) / (vol*np.sqrt(252))
print('Sharpe Ratio assumming IID returns: ',sharpe_ratio)
print('Autocorrelation: ', total_account_balance.pct_change().fillna(0).autocorr(lag=1))
# accounting for non-zero autocorrelatio, daily sr should be calculated as:
# the daily sharpe ratio is then multiplied by the annualization factor proposed by the paper: The
# Statistics of Sharpe Ratios by Andrew W Lo
annualized_ret = total_account_balance.pct_change().fillna(0).mean()
rf_daily = (1+rf)**(1/252)-1
sharpe_ratio = (annualized_ret-rf_daily) /vol
print('Daily Sharpe Ratio', sharpe_ratio)

print(sharpe_ratio)
'''

"from datetime import timedelta\n\npairs = total_pairs\n\n\n# calculate total daily account balance & df with returns\ntotal_account_balance = performance[0][1]['account_balance'].resample('D').last().dropna()\nportfolio_returns = total_account_balance.pct_change().fillna(0)\nfor index in range(1, len(pairs)):\n    pair_balance = performance[index][1]['account_balance'].resample('D').last().dropna()\n    total_account_balance = total_account_balance + pair_balance\n    print(total_account_balance)  #ERRO\n    portfolio_returns = pd.concat([portfolio_returns, pair_balance.pct_change().fillna(0)], axis=1)\n\n\n# add first day with initial balance\ntotal_account_balance = pd.Series(data=[len(pairs)],\n                                  index=[total_account_balance.index[0] - timedelta(days=1)]).append(\n                                  total_account_balance)\n\n# calculate portfolio volatility\nweights = np.array([1 / len(pairs)] * len(pairs))\nvol = np.sqrt(np.dot(weights.T, np.dot(portf

In [ ]:
performance_threshold_test_t[0][1]

IndexError: list index out of range

# Transações

In [ ]:
'''
results, pairs_summary = trader.summarize_results(sharpe_results_threshold_test_w_costs,
                                                  cum_returns_threshold_test_w_costs,
                                                  performance_threshold_test,
                                                  pairs_all_a_all,
                                                  n_years_test,
                                                  rf = rfannualized)
'''

In [ ]:
print('Corrected Annualized SR: {:.2f}'.format(0.3629691498850566*(252**0.5)))

In [ ]:
'''
df = pd.DataFrame(results, index=[0])
df = df.append(results,ignore_index=True)
'''